In [5]:
import sys
import os
sys.path.append('..')

from Utils.TinyImageNet_loader import get_tinyimagenet_dataloaders
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from timm.models import swin_transformer
import timm
from torch.utils.data import DataLoader

# Hyperparameters
batch_size = 64
image_size = 224  # Swin Transformer input image size
num_classes = 200  # Tiny ImageNet classes
epochs = 100
learning_rate = 0.001
weight_decay = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to define data loaders
def get_tinyimagenet_dataloaders(data_dir, transform_train, transform_val, transform_test, batch_size, image_size):
    train_dir = f"{data_dir}/train"
    val_dir = f"{data_dir}/val"
    test_dir = f"{data_dir}/test"

    # Use ImageFolder for dataset loading
    train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
    val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

# Preprocessing transformations
tiny_transform_train = transforms.Compose([
    transforms.RandomResizedCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

tiny_transform_val = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

tiny_transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset directory
data_dir = '../datasets'

# Data loaders
train_loader, val_loader, test_loader = get_tinyimagenet_dataloaders(
    data_dir=data_dir,
    transform_train=tiny_transform_train,
    transform_val=tiny_transform_val,
    transform_test=tiny_transform_test,
    batch_size=batch_size,
    image_size=image_size,
)

# Define Swin Transformer model
class SwinTransformerTinyImageNet(nn.Module):
    def __init__(self, num_classes):
        super(SwinTransformerTinyImageNet, self).__init__()
        self.backbone = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=num_classes)
    
    def forward(self, x):
        return self.backbone(x)

# Count the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Function to calculate top-k accuracy
def calculate_top_k_accuracy(outputs, labels, k=1):
    _, top_k_predictions = outputs.topk(k, dim=1, largest=True, sorted=True)
    correct = top_k_predictions.eq(labels.view(-1, 1).expand_as(top_k_predictions))
    return correct.sum().item()

# Function to calculate top-1, top-3, top-5 accuracies
def calculate_accuracies(outputs, labels):
    top_1_acc = calculate_top_k_accuracy(outputs, labels, k=1)
    top_3_acc = calculate_top_k_accuracy(outputs, labels, k=3)
    top_5_acc = calculate_top_k_accuracy(outputs, labels, k=5)
    return top_1_acc, top_3_acc, top_5_acc

# Training and evaluation loop
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs, device):
    best_accuracy = 0.0

    for epoch in range(epochs):
        # Training phase
        model.train()
        total_samples = 0
        top_1_correct = 0
        top_3_correct = 0
        top_5_correct = 0

        for batch in train_loader:
            inputs, labels = batch  # Ensure proper unpacking
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            top_1, top_3, top_5 = calculate_accuracies(outputs, labels)
            total_samples += labels.size(0)
            top_1_correct += top_1
            top_3_correct += top_3
            top_5_correct += top_5

        train_top1_acc = 100. * top_1_correct / total_samples
        train_top3_acc = 100. * top_3_correct / total_samples
        train_top5_acc = 100. * top_5_correct / total_samples
        print(f"Epoch {epoch+1} Train Acc (Top-1): {train_top1_acc:.2f}%, Top-3: {train_top3_acc:.2f}%, Top-5: {train_top5_acc:.2f}%")

        # Validation phase
        model.eval()
        total_samples = 0
        top_1_correct = 0
        top_3_correct = 0
        top_5_correct = 0

        with torch.no_grad():
            for batch in test_loader:
                inputs, labels = batch  # Ensure proper unpacking
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)

                top_1, top_3, top_5 = calculate_accuracies(outputs, labels)
                total_samples += labels.size(0)
                top_1_correct += top_1
                top_3_correct += top_3
                top_5_correct += top_5

        test_top1_acc = 100. * top_1_correct / total_samples
        test_top3_acc = 100. * top_3_correct / total_samples
        test_top5_acc = 100. * top_5_correct / total_samples
        print(f"Epoch {epoch+1} Test Acc (Top-1): {test_top1_acc:.2f}%, Top-3: {test_top3_acc:.2f}%, Top-5: {test_top5_acc:.2f}%")

        # Save the best model
        if test_top1_acc > best_accuracy:
            best_accuracy = test_top1_acc
            torch.save(model.state_dict(), 'best_swin_transformer.pth')
            print("Model saved!")

        scheduler.step()

    print(f"Best Test Accuracy (Top-1): {best_accuracy:.2f}%")
    return model

# Initialize the model
model = SwinTransformerTinyImageNet(num_classes=num_classes).to(device)
num_params = count_parameters(model)
print(f"Number of Parameters: {num_params}")

# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# Train and evaluate the model
trained_model = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs, device)

# Load the best model and test
model.load_state_dict(torch.load('best_swin_transformer.pth'))
trained_model.eval()


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../datasets/train'

: 